In [7]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# File paths
DATA_PATH = "/Users/fathhi/Desktop/Interview Tasks/Air_Quality_Analysis/data/"

hourly_path = DATA_PATH + "df_hourly_firmware_status.csv"             
ref_path    = DATA_PATH + "earthsense_reference.csv"  
meta_path   = DATA_PATH + "earthsense_metadata.csv"   
change_path = DATA_PATH + "earthsense_changelog.csv"  

In [8]:
# Load main datasets
df_hourly = pd.read_csv(hourly_path, parse_dates=["timestamp_recorded_utc"])
df_ref    = pd.read_csv(ref_path, parse_dates=["timestamp_utc"])
df_meta   = pd.read_csv(meta_path)
df_change = pd.read_csv(change_path)

print("Hourly dataset shape :", df_hourly.shape)
print("Reference dataset shape:", df_ref.shape)
print("Metadata shape:", df_meta.shape)
print("Changelog shape:", df_change.shape)


Hourly dataset shape : (22560, 21)
Reference dataset shape: (1128, 6)
Metadata shape: (20, 7)
Changelog shape: (3, 4)


In [9]:
df_ref = df_ref.rename(columns={
    "temp_c": "ref_temp_c",
    "rh_pct": "ref_rh_pct"
})

df_ref.head()

,timestamp_utc,ref_no2_ppb,ref_pm25_ugm3,ref_temp_c,ref_rh_pct,ref_no2_ugm3
0,2025-06-15 00:00:00+00:00,26.95,8.20,13.55,77.04,52.70
1,2025-06-15 01:00:00+00:00,24.74,8.96,10.01,83.62,48.98
2,2025-06-15 02:00:00+00:00,24.83,10.43,12.55,84.92,48.73
3,2025-06-15 03:00:00+00:00,21.64,8.79,12.66,91.96,42.45
4,2025-06-15 04:00:00+00:00,21.56,9.96,12.40,87.99,42.33


In [10]:
# Merge df_hourly with reference on timestamp
df_valid = pd.merge(
    df_hourly,
    df_ref,
    left_on="timestamp_recorded_utc",
    right_on="timestamp_utc",
    how="inner"
)

print("Validation dataset shape:", df_valid.shape)
df_valid.head()


Validation dataset shape: (22560, 27)


,sensor_id,timestamp_recorded_utc,no2_ppb,no2_ugm3,pm25_ugm3,temp_c,rh_pct,flag_out_of_range__no2_ppb,flag_out_of_range__no2_ugm3,flag_out_of_range__pm25_ugm3,...,flag_missing__rh_pct,flag_missing_any,firmware,status,timestamp_utc,ref_no2_ppb,ref_pm25_ugm3,ref_temp_c,ref_rh_pct,ref_no2_ugm3
0,ZEPHYR_001,2025-06-15 00:00:00+00:00,25.20,49.47,6.67,12.42,77.34,0.0,0.0,0.0,...,0.0,0.0,1.4.2,OK,2025-06-15 00:00:00+00:00,26.95,8.20,13.55,77.04,52.70
1,ZEPHYR_001,2025-06-15 01:00:00+00:00,22.16,43.93,8.62,9.66,81.89,0.0,0.0,0.0,...,0.0,0.0,1.4.2,OK,2025-06-15 01:00:00+00:00,24.74,8.96,10.01,83.62,48.98
2,ZEPHYR_001,2025-06-15 02:00:00+00:00,20.43,40.33,11.01,10.89,85.90,0.0,0.0,0.0,...,0.0,0.0,1.4.2,OK,2025-06-15 02:00:00+00:00,24.83,10.43,12.55,84.92,48.73
3,ZEPHYR_001,2025-06-15 03:00:00+00:00,17.73,34.76,2.65,12.80,98.05,0.0,0.0,0.0,...,0.0,0.0,1.4.2,OK,2025-06-15 03:00:00+00:00,21.64,8.79,12.66,91.96,42.45
4,ZEPHYR_001,2025-06-15 04:00:00+00:00,19.32,37.87,14.30,12.89,91.62,0.0,0.0,0.0,...,0.0,0.0,1.4.2,OK,2025-06-15 04:00:00+00:00,21.56,9.96,12.40,87.99,42.33


In [11]:
df_valid = df_valid.drop(columns=["timestamp_utc"])

In [12]:
df_hourly.columns

Index(['sensor_id', 'timestamp_recorded_utc', 'no2_ppb', 'no2_ugm3',
       'pm25_ugm3', 'temp_c', 'rh_pct', 'flag_out_of_range__no2_ppb',
       'flag_out_of_range__no2_ugm3', 'flag_out_of_range__pm25_ugm3',
       'flag_out_of_range__rh_pct', 'flag_out_of_range__temp_c',
       'flag_outlier_any', 'flag_missing__no2_ppb', 'flag_missing__no2_ugm3',
       'flag_missing__pm25_ugm3', 'flag_missing__temp_c',
       'flag_missing__rh_pct', 'flag_missing_any', 'firmware', 'status'],
      dtype='object')

In [13]:
df_valid.columns

Index(['sensor_id', 'timestamp_recorded_utc', 'no2_ppb', 'no2_ugm3',
       'pm25_ugm3', 'temp_c', 'rh_pct', 'flag_out_of_range__no2_ppb',
       'flag_out_of_range__no2_ugm3', 'flag_out_of_range__pm25_ugm3',
       'flag_out_of_range__rh_pct', 'flag_out_of_range__temp_c',
       'flag_outlier_any', 'flag_missing__no2_ppb', 'flag_missing__no2_ugm3',
       'flag_missing__pm25_ugm3', 'flag_missing__temp_c',
       'flag_missing__rh_pct', 'flag_missing_any', 'firmware', 'status',
       'ref_no2_ppb', 'ref_pm25_ugm3', 'ref_temp_c', 'ref_rh_pct',
       'ref_no2_ugm3'],
      dtype='object')

In [14]:
fw_versions = df_hourly.groupby("sensor_id")["firmware"].unique()
print(fw_versions)

sensor_id
ZEPHYR_001    [1.4.2, 1.5.0]
ZEPHYR_002           [1.4.2]
ZEPHYR_003           [1.4.2]
ZEPHYR_004    [1.4.2, 1.5.0]
ZEPHYR_005    [1.4.2, 1.5.0]
ZEPHYR_006           [1.4.2]
ZEPHYR_007    [1.4.2, 1.5.0]
ZEPHYR_008    [1.4.2, 1.5.0]
ZEPHYR_009    [1.4.2, 1.5.0]
ZEPHYR_010           [1.4.2]
ZEPHYR_011           [1.4.2]
ZEPHYR_012           [1.4.2]
ZEPHYR_013    [1.4.2, 1.5.0]
ZEPHYR_014           [1.4.2]
ZEPHYR_015      [1.4.2, nan]
ZEPHYR_016           [1.4.2]
ZEPHYR_017           [1.4.2]
ZEPHYR_018    [1.4.2, 1.5.0]
ZEPHYR_019    [1.4.2, 1.5.0]
ZEPHYR_020           [1.4.2]
Name: firmware, dtype: object


In [19]:
print(df_valid.columns.tolist())


['sensor_id', 'timestamp_recorded_utc', 'no2_ppb', 'no2_ugm3', 'pm25_ugm3', 'temp_c', 'rh_pct', 'flag_out_of_range__no2_ppb', 'flag_out_of_range__no2_ugm3', 'flag_out_of_range__pm25_ugm3', 'flag_out_of_range__rh_pct', 'flag_out_of_range__temp_c', 'flag_outlier_any', 'flag_missing__no2_ppb', 'flag_missing__no2_ugm3', 'flag_missing__pm25_ugm3', 'flag_missing__temp_c', 'flag_missing__rh_pct', 'flag_missing_any', 'firmware', 'status', 'ref_no2_ppb', 'ref_pm25_ugm3', 'ref_temp_c', 'ref_rh_pct', 'ref_no2_ugm3']


In [26]:
df_valid["sensor_id"].unique()


array(['ZEPHYR_001', 'ZEPHYR_002', 'ZEPHYR_003', 'ZEPHYR_004',
       'ZEPHYR_005', 'ZEPHYR_006', 'ZEPHYR_007', 'ZEPHYR_008',
       'ZEPHYR_009', 'ZEPHYR_010', 'ZEPHYR_011', 'ZEPHYR_012',
       'ZEPHYR_013', 'ZEPHYR_014', 'ZEPHYR_015', 'ZEPHYR_016',
       'ZEPHYR_017', 'ZEPHYR_018', 'ZEPHYR_019', 'ZEPHYR_020'],
      dtype=object)

In [ ]:
# --- Step 1. Define firmware update date ---
firmware_date = "2025-07-10"

# --- Step 2. Compute NO2 bias (sensor - reference) ---
df_valid["no2_bias"] = df_valid["no2_ppb"] - df_valid["ref_no2_ppb"]

# --- Step 3. Split into pre and post firmware update ---
pre = df_valid[df_valid["timestamp_recorded_utc"] < firmware_date]
post = df_valid[df_valid["timestamp_recorded_utc"] >= firmware_date]

# --- Step 4. Calculate mean bias per sensor before/after ---
bias_pre = pre.groupby("sensor_id")["no2_bias"].mean()
bias_post = post.groupby("sensor_id")["no2_bias"].mean()

# --- Step 5. Combine into one summary table ---
bias_table = pd.concat([bias_pre, bias_post], axis=1)
bias_table.columns = ["Bias_Pre", "Bias_Post"]

# Bias shift = Post - Pre
bias_table["Bias_Shift"] = bias_table["Bias_Post"] - bias_table["Bias_Pre"]

# --- Step 6. Flag affected sensors (large positive jump) ---
bias_table["Affected"] = bias_table["Bias_Shift"].apply(lambda x: "Yes" if x > 3 else "No")

# --- Final result ---
print(bias_table)


            Bias_Pre  Bias_Post  Bias_Shift Affected
sensor_id                                           
ZEPHYR_001 -0.245866   8.928106    9.173972      Yes
ZEPHYR_002 -1.911404  -2.032538   -0.121134       No
ZEPHYR_003 -1.775367  -1.820486   -0.045120       No
ZEPHYR_004  8.079019  16.903447    8.824428      Yes
ZEPHYR_005 -2.075633   6.737452    8.813085      Yes
ZEPHYR_006 -1.930473  -2.111875   -0.181402       No
ZEPHYR_007 -0.054283   4.094012    4.148295      Yes
ZEPHYR_008  8.131117  11.795415    3.664298      Yes
ZEPHYR_009  0.115395   4.097386    3.981991      Yes
ZEPHYR_010  8.123460   8.056875   -0.066585       No
ZEPHYR_011  8.046437  12.637178    4.590741      Yes
ZEPHYR_012  8.190717   7.871811   -0.318906       No
ZEPHYR_013 -1.842616   1.871155    3.713771      Yes
ZEPHYR_014  0.094128   0.090133   -0.003995       No
ZEPHYR_015 -2.001256  -2.057557   -0.056300       No
ZEPHYR_016 -2.061483  -2.021680    0.039803       No
ZEPHYR_017  0.082333  -0.084475   -0.166809   